# Import packages

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import time
import numpy as np
import pandas as pd

In [ ]:
import tweepy

# Obtain key codes to access Twitter's API

Sign in with your existing Twitter account (or make a new one) at https://twitter.com/. Go to apps.twitter.com. Select "Create new app" to make a new application. Fill out the form and feel free to use very generic information (i.e. - "http://www.darden.virginia.edu/" as your website). You can ignore the callback URL field.

Go to Keys and Access Tokens tab. After submitting, you should have the option to "Create Access Token". This is how you actually get to access the API. For access type, "read only" should suffice, because you are not using the code to write tweets back to the site. You will get OAuth settings, which you should save in a safe place.

Put your codes in place of xxx in "xxx" below. Be sure to keep the quotation marks.

In [ ]:
consumer_key = "xxxx"
consumer_secret = "xxxx"
access_token = "xxxx"
access_token_secret = "xxxx"

The cell below will provide access to Twitter's API. Note that we have changed our "wait on rate limit" settings because we will be pulling down a fair amount of information from Twitter.

In [ ]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, compression=True)

# Obtain User's Followers

In [ ]:
startingScreenName = "ChrisGrant360"

Use Tweepy to translate this user's screen name into her ID on Twitter. Use the ID to pull each one of her follower's screen names and store them in the list called followerScreenNames. 

The try and except lines are used to handle exceptions when a follower is private or has been suspended. The code skips over those users and continues the loop. 

Note that you may reach a rate limit when running this loop. Twitter will put your loop to sleep for a few minutes and resume later automatically.  This loop will take about 15 minutes to complete.  

In [ ]:
start_time = time.time()
followerScreenNames = []
for fid in tweepy.Cursor(api.followers_ids, screen_name=startingScreenName).items():
    try:
        fsn = api.get_user(id=fid).screen_name
        followerScreenNames.append(fsn)
    except:
        pass
time.time() - start_time

Store the relationship between the user and her followers in a master data frame (master_df). In a network, there is typically a source and a target which represents where the relationship is originating from and where it ends. In this case, the startingScreenName is the source and the followerScreenNames that we pulled from Twitter is the target. We store these in two columns in master_df called 'source' and 'target' and print master_df to check the data frame.

In [ ]:
source_list = np.repeat(startingScreenName, len(followerScreenNames))
target_list = followerScreenNames
master_df = pd.DataFrame()
master_df['source'] = source_list
master_df['target'] = target_list
master_df

How many followers does she have?

# Obtain Followers of User's Followers

To fill out the network, we want to pull followers of the targets (or the followers of the user's followers), starting with the first target, i.e., the screen name in the first row, second column in master_df above. The .ix notation after master_df allows you to choose which row and column to start from. Similar to the code above, we use tweepy to obtain the followers of a user. We then store the information in two columns (again source and target) in an intermediate dataframe and append it to the master data frame.

WARNING: DO NOT RUN THE LOOP BELOW UNLESS YOU PLAN TO LET IT RUN FOR SEVERAL HOURS. SKIP DOWN TWO CELLS TO LOAD IN THE RESULTING DATA.

In [ ]:
start_time = time.time()
for i in range(0, len(followerScreenNames)):
    this_screen_name = master_df.ix[i, 'target']
    followerScreenNames1 = []
    try:
        for fid in tweepy.Cursor(api.followers_ids, screen_name=this_screen_name).items():  
            fsn = api.get_user(id=fid).screen_name
            followerScreenNames1.append(fsn)
    except Exception:
        pass
    intermediate_df = pd.DataFrame(columns=['source', 'target'])    
    intermediate_df['source'] = np.repeat(this_screen_name, len(followerScreenNames1))
    intermediate_df['target'] = followerScreenNames1
    master_df = pd.concat([master_df, intermediate_df])
    master_df = master_df.reset_index(drop=True)
time.time() - start_time

If you finish the loop, write the resulting data frame to a text file and a csv file (to be used in Gephi).

In [ ]:
master_df.tail()

In [ ]:
master_df.to_csv('ChrisGrant360Edges.csv', header=False, index=False, sep=',')

SKIP DOWN TO HERE. Load the data with the user's followers and the followers of the user's followers.  Look at the resulting data frame.

In [ ]:
master_df2 = pd.read_csv('ChrisGrant360Edges.csv', header=None)
master_df2